# Case Preparation

In [1]:
import numpy as np
import pandas as pd

import andes
import ams

In [2]:
andes.config_logger(stream_level=30)
ams.config_logger(stream_level=30)

In [3]:
sp = ams.load('./../cases/IL200_opf.xlsx',
               setup=True, no_output=True,
               default_config=True)

In [4]:
stgs = sp.StaticGen.find_idx(keys='gentype',
                             values=['W2', 'PV', 'ES'],
                             allow_all=True)

for stg in stgs:
    gcost = sp.GCost.find_idx(keys='gen', values=stg)
    sp.GCost.alter(src='c2', attr='vin', idx=gcost, value=0)
    sp.GCost.alter(src='c1', attr='vin', idx=gcost, value=0)
    sp.GCost.alter(src='c0', attr='vin', idx=gcost, value=0)

    sp.StaticGen.set(src='ctrl', attr='vin', idx=stg, value=0)

stgs = sp.StaticGen.find_idx(keys='genfuel',
                             values=['coal', 'nuclear'],
                             allow_all=True)
for stg in stgs:
    sp.StaticGen.alter(src='pmin', attr='vin', idx=stg, value=0)

slack = sp.Slack.idx.v
slack_gcost = sp.GCost.find_idx(keys='gen', values=slack)

pmax0 = sp.StaticGen.get(src='pmax', attr='v', idx=slack)
sp.StaticGen.alter(src='pmax', attr='vin', idx=slack, value=5*pmax0)

for param in ['c2', 'c1', 'c0']:
    v0 = sp.GCost.get(attr='v', idx=slack_gcost, src=param)
    sp.GCost.alter(src=param, attr='vin', idx=slack_gcost, value=10*v0)

sp.to_xlsx('./../cases/IL200_opf2.xlsx', overwrite=True)

True

In [5]:
stg_idx = sp.StaticGen.get_all_idxes()
stg_wt = sp.StaticGen.find_idx(keys='genfuel', values=['wind'], allow_all=True)[0]
stg_pv = sp.StaticGen.find_idx(keys='genfuel', values=['solar'], allow_all=True)[0]
stg_es = sp.StaticGen.find_idx(keys='genfuel', values=['ess'], allow_all=True)[0]

sn_wt = sp.StaticGen.get(src='Sn', attr='v', idx=stg_wt)
sn_pv = sp.StaticGen.get(src='Sn', attr='v', idx=stg_pv)
sn_es = sp.StaticGen.get(src='Sn', attr='v', idx=stg_es)
sn_total = sp.StaticGen.get(src='Sn', attr='v', idx=stg_idx)

print('Total capacity: {0:.2f} MVA'.format(sn_total.sum()))
print('Wind capacity: {0:.2f} MVA'.format(sn_wt.sum()))
print('Solar capacity: {0:.2f} MVA'.format(sn_pv.sum()))
print('ESS capacity: {0:.2f} MVA'.format(sn_es.sum()))

Total capacity: 4251.60 MVA
Wind capacity: 1040.52 MVA
Solar capacity: 1916.70 MVA
ESS capacity: 36.72 MVA


## Inspect Case

There are different types of generators:
- ST: Steam Turbine (includes nuclear, geothermal, and solar steam)
- NB: ST - Boiling Water Nuclear Reactor
- W2: Wind Turbine, Type 2
- GT: Combustion (Gas) Turbine
- PV: Photovoltaic
- ESS: Energy Storage System

In [6]:
stg_dfs = sp.StaticGen.as_df()
stg_cols = ['idx', 'bus', 'Sn', 'gentype', 'genfuel']

stg_df = stg_dfs[stg_cols]

stg_df

,idx,bus,Sn,gentype,genfuel
0,47,189,682.98,NB,nuclear
1,1,49,5.44,ST,coal
2,2,50,5.44,ST,coal
3,3,51,5.44,ST,coal
4,4,52,5.44,ST,coal
5,5,53,10.88,ST,coal
6,6,65,180.48,W2,wind
7,7,67,5.64,ST,coal
8,8,68,33.50,W2,wind
9,9,69,33.50,W2,wind


## Dynamic Models Replacement

In the original dynamic case, only ``GENROU`` is used for dynamic generators.
To better represent the dynamic behavior, following replacements are made:
1. W2 type gen, replace GENROU with: ``PVD1``
1. PV type gen, replace GENROU with: ``PVD1``
1. ESS type gen, replace GENROU with ``ESD1``

In [7]:
# load the dynamics case but don't set up it
dyn_base_case = './../cases/IL200_dyn_new.xlsx'

sa = andes.load(dyn_base_case,
                setup=False, no_output=True,
                default_config=True)

# NOTE: bias is manually measured, in unit MW/0.1Hz
slack_bus = sa.Slack.bus.v[0]
sa.add('ACEc', param_dict=dict(bus=slack_bus, bias=-45))

Generating code for 1 models on 12 processes.


'ACEc_1'

In [8]:
# In the ANDES case, get the StaticGen idx for type W2, NB, ST, GT, respectively
stg_idxes = sp.StaticGen.find_idx(keys='gentype',
                                  values=['W2', 'PV', 'ES'],
                                  allow_all=True)
stg_w2t = stg_idxes[0]
stg_pv = stg_idxes[1]
stg_ess = stg_idxes[2]

n_w2t = len(stg_w2t)
n_pv = len(stg_pv)
n_ess = len(stg_ess)

# In the ANDES case, get the corresponding SynGen idx
syg_w2t = sa.SynGen.find_idx(keys='gen', values=stg_w2t)
syg_pv = sa.SynGen.find_idx(keys='gen', values=stg_pv)
syg_ess = sa.SynGen.find_idx(keys='gen', values=stg_ess)

In [9]:
# --- GENROU -> WT2G: REGF2WT ---
REGF2WT = pd.DataFrame()

# mapped parameters
REGF2WT['u'] = sa.SynGen.get(src='u', idx=syg_w2t)
REGF2WT['idx'] = [f'WT_{i}' for i in range(1, n_w2t + 1)]
REGF2WT['bus'] = sa.SynGen.get(src='bus', idx=syg_w2t)
REGF2WT['gen'] = sa.SynGen.get(src='gen', idx=syg_w2t)
REGF2WT['Sn'] = sa.SynGen.get(src='Sn', idx=syg_w2t)
REGF2WT['PQFLAG'] = [1] * n_w2t  # P priority
REGF2WT['fn'] = [sa.config.freq] * n_w2t
REGF2WT['dwmax'] = [75] * n_w2t
REGF2WT['dwmin'] = [-75] * n_w2t
REGF2WT['wdrp'] = [0.033] * n_w2t
REGF2WT['Qdrp'] = [0.045] * n_w2t
REGF2WT['KPi'] = [500] * n_w2t
REGF2WT['KIi'] = [0] * n_w2t
REGF2WT['Pmax'] = [999] * n_w2t
REGF2WT['Pmin'] = [-999] * n_w2t
REGF2WT['Qmax'] = [999] * n_w2t
REGF2WT['Qmin'] = [-999] * n_w2t
REGF2WT['gammap'] = sa.SynGen.get(src='gammap', idx=syg_w2t)
REGF2WT['gammaq'] = sa.SynGen.get(src='gammaq', idx=syg_w2t)

# unmapped parameters are skipped and default values are used
# add to the system
regf2_w2t = []
for row in REGF2WT.itertuples(index=False):
    idx = sa.add(model='REGF2', param_dict={**row._asdict()})
    regf2_w2t.append(idx)

# turn off the original GENROU
sa.SynGen.alter(src='u', value=0, idx=syg_w2t)

True

In [10]:
# --- GENROU -> PV: REGF2PV ---
REGF2PV = pd.DataFrame()

# mapped parameters
REGF2PV['u'] = sa.SynGen.get(src='u', idx=syg_pv)
REGF2PV['idx'] = [f'PV_{i}' for i in range(1, n_pv + 1)]
REGF2PV['bus'] = sa.SynGen.get(src='bus', idx=syg_pv)
REGF2PV['gen'] = sa.SynGen.get(src='gen', idx=syg_pv)
REGF2PV['Sn'] = sa.SynGen.get(src='Sn', idx=syg_pv)
REGF2PV['PQFLAG'] = [1] * n_pv  # P priority
REGF2PV['fn'] = [sa.config.freq] * n_pv
REGF2PV['dwmax'] = [75] * n_pv
REGF2PV['dwmin'] = [-75] * n_pv
REGF2PV['wdrp'] = [0.033] * n_pv
REGF2PV['Qdrp'] = [0.045] * n_pv
REGF2PV['KPi'] = [500] * n_pv
REGF2PV['KIi'] = [0] * n_pv
REGF2PV['Pmax'] = [999] * n_pv
REGF2PV['Pmin'] = [-999] * n_pv
REGF2PV['Qmax'] = [999] * n_pv
REGF2PV['Qmin'] = [-999] * n_pv
REGF2PV['gammap'] = sa.SynGen.get(src='gammap', idx=syg_pv)
REGF2PV['gammaq'] = sa.SynGen.get(src='gammaq', idx=syg_pv)

# unmapped parameters are skipped and default values are used
# add to the system
regf2_pv = []
for row in REGF2PV.itertuples(index=False):
    idx = sa.add(model='REGF2', param_dict={**row._asdict()})
    regf2_pv.append(idx)

# turn off the original GENROU
sa.SynGen.alter(src='u', value=0, idx=syg_pv)

True

In [11]:
# --- GENROU -> ESS: REGF2ES ---
REGF2ES = pd.DataFrame()

# mapped parameters
REGF2ES['u'] = sa.SynGen.get(src='u', idx=syg_ess)
REGF2ES['idx'] = [f'ESS_{i}' for i in range(1, n_ess + 1)]
REGF2ES['bus'] = sa.SynGen.get(src='bus', idx=syg_ess)
REGF2ES['gen'] = sa.SynGen.get(src='gen', idx=syg_ess)
REGF2ES['Sn'] = sa.SynGen.get(src='Sn', idx=syg_ess)
REGF2ES['PQFLAG'] = [1] * n_ess  # P priority
REGF2ES['fn'] = [sa.config.freq] * n_ess
REGF2ES['dwmax'] = [75] * n_ess
REGF2ES['dwmin'] = [-75] * n_ess
REGF2ES['wdrp'] = [0.033] * n_ess
REGF2ES['Qdrp'] = [0.045] * n_ess
REGF2ES['KPi'] = [500] * n_ess
REGF2ES['KIi'] = [0] * n_ess
REGF2ES['Pmax'] = [999] * n_ess
REGF2ES['Pmin'] = [-999] * n_ess
REGF2ES['Qmax'] = [999] * n_ess
REGF2ES['Qmin'] = [-999] * n_ess
REGF2ES['gammap'] = sa.SynGen.get(src='gammap', idx=syg_ess)
REGF2ES['gammaq'] = sa.SynGen.get(src='gammaq', idx=syg_ess)

# unmapped parameters are skipped and default values are used
# add to the system
regf2_es = []
for row in REGF2ES.itertuples(index=False):
    idx = sa.add(model='REGF2', param_dict={**row._asdict()})
    regf2_es.append(idx)

# turn off the original GENROU
sa.SynGen.alter(src='u', value=0, idx=syg_ess)

True

In [12]:
sa.setup()

# relax TurbineGov upper limit
vmax0 = sa.TGOV1NDB.get(src='VMAX', attr='v', idx=sa.TGOV1NDB.idx.v)
sa.TGOV1NDB.set(src='VMAX', attr='v', idx=sa.TGOV1NDB.idx.v,
                value=100 * vmax0)

True

In [13]:
sa.PFlow.run()
_ = sa.TDS.init()

GENROU (vf range) out of typical lower limit.

   idx     | values | limit
-----------+--------+------
 GENROU_6  | 0      | 1    
 GENROU_7  | 0.865  | 1    
 GENROU_8  | 0      | 1    
 GENROU_9  | 0      | 1    
 GENROU_10 | 0      | 1    
 GENROU_11 | 0      | 1    
 GENROU_12 | 0      | 1    
 GENROU_13 | 0      | 1    
 GENROU_14 | 0      | 1    
 GENROU_22 | 0      | 1    
 GENROU_23 | 0      | 1    
 GENROU_24 | 0      | 1    
 GENROU_25 | 0      | 1    
 GENROU_26 | 0      | 1    
 GENROU_27 | 0      | 1    
 GENROU_28 | 0      | 1    
 GENROU_29 | 0      | 1    
 GENROU_30 | 0      | 1    
 GENROU_31 | 0      | 1    
 GENROU_32 | 0      | 1    
 GENROU_33 | 0      | 1    
 GENROU_34 | 0      | 1    
 GENROU_35 | 0      | 1    
 GENROU_36 | 0      | 1    
 GENROU_38 | 0.977  | 1    
 GENROU_46 | 0      | 1    




In [14]:
sa.TDS.config.no_tqdm = True

sa.TDS.run()

True

In [15]:
# Export to XLSX for Further Use
andes.io.xlsx.write(sa, './../cases/IL200_dyn_db.xlsx',
                    overwrite=True)

True

In [16]:
genrou_off = sa.GENROU.find_idx(keys='u', values=[0], allow_all=True)[0]
tgov1_off = sa.TGOV1NDB.find_idx(keys='syn', values=genrou_off)
sexs_off = sa.SEXS.find_idx(keys='syn', values=genrou_off)

genrou_off_uid = sa.GENROU.idx2uid(genrou_off)
tgov1_off_uid = sa.TGOV1NDB.idx2uid(tgov1_off)
sexs_off_uid = sa.SEXS.idx2uid(sexs_off)

In the code block below, offline devices in GENROU, TGOV1NDB, and SEXS are deleted.

In [17]:
xls0 = pd.ExcelFile('./../cases/IL200_dyn_db.xlsx')

revised_sheets = dict()

for sheet_name in xls0.sheet_names:
    if sheet_name == 'GENROU':
        df = pd.read_excel(xls0, sheet_name=sheet_name)
        df = df.drop(genrou_off_uid, axis=0)
        revised_sheets[sheet_name] = df
    elif sheet_name == 'TGOV1NDB':
        df = pd.read_excel(xls0, sheet_name=sheet_name)
        df = df.drop(tgov1_off_uid, axis=0)
        revised_sheets[sheet_name] = df
    elif sheet_name == 'SEXS':
        df = pd.read_excel(xls0, sheet_name=sheet_name)
        df = df.drop(sexs_off_uid, axis=0)
        revised_sheets[sheet_name] = df
    else:
        revised_sheets[sheet_name] = pd.read_excel(xls0, sheet_name=sheet_name)

with pd.ExcelWriter("./../cases/IL200_dyn_db2.xlsx", engine="openpyxl") as writer:
    for sheet_name, df in revised_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name,
                    index=False, freeze_panes=(1, 0))

In [18]:
sa0 = andes.load('./../cases/IL200_dyn_db2.xlsx',
                 setup=True, no_output=True,
                 default_config=True)

sa0.PFlow.run()
sa0.TDS.init()

sa0.EIG.run()
sa0.EIG.report()

Generating code for 1 models on 12 processes.


GENROU (vf range) out of typical lower limit.

   idx     | values | limit
-----------+--------+------
 GENROU_7  | 0.865  | 1    
 GENROU_38 | 0.977  | 1    


